In [1]:
import pandas as pd
import numpy as np
import csv
import os
from datetime import datetime, timedelta
from dateutil.parser import parse, tz
from dateutil.relativedelta import relativedelta

In [23]:
# Select sites to extract
baaqmdsites = np.array(["Laney College","San Pablo - Rumrill","Oakland West"])
baaqmdcodes = np.array(["BQL_","BQS_","BQO_"])

# Set start and end dates for data to process
start_date = '2017-01-01' # format YYYY-MM-DD
start_datetime = datetime(2017,1,1)
end_date = '2018-01-01' #Day after last day of processed data, format YYYY-MM-DD
end_datetime = datetime(2018,1,1) #Day after last day of processed data

In [3]:
import sys
import os
import re
import fnmatch
try:
    _unicode = unicode
except NameError:
    # If Python is built without Unicode support, the unicode type
    # will not exist. Fake one.
    class _unicode(object):
        pass
__all__ = ["glob", "iglob"]
def glob(pathname):
    """Return a list of paths matching a pathname pattern.

    The pattern may contain simple shell-style wildcards a la
    fnmatch. However, unlike fnmatch, filenames starting with a
    dot are special cases that are not matched by '*' and '?'
    patterns.

    """
    return list(iglob(pathname))
def iglob(pathname):
    """Return an iterator which yields the paths matching a pathname pattern.

    The pattern may contain simple shell-style wildcards a la
    fnmatch. However, unlike fnmatch, filenames starting with a
    dot are special cases that are not matched by '*' and '?'
    patterns.

    """
    dirname, basename = os.path.split(pathname)
    if not has_magic(pathname):
        if basename:
            if os.path.lexists(pathname):
                yield pathname
        else:
            # Patterns ending with a slash should match only directories
            if os.path.isdir(dirname):
                yield pathname
        return
    if not dirname:
        for name in glob1(os.curdir, basename):
            yield name
        return
    # `os.path.split()` returns the argument itself as a dirname if it is a
    # drive or UNC path.  Prevent an infinite recursion if a drive or UNC path
    # contains magic characters (i.e. r'\\?\C:').
    if dirname != pathname and has_magic(dirname):
        dirs = iglob(dirname)
    else:
        dirs = [dirname]
    if has_magic(basename):
        glob_in_dir = glob1
    else:
        glob_in_dir = glob0
    for dirname in dirs:
        for name in glob_in_dir(dirname, basename):
            yield os.path.join(dirname, name)

# These 2 helper functions non-recursively glob inside a literal directory.
# They return a list of basenames. `glob1` accepts a pattern while `glob0`
# takes a literal basename (so it only has to check for its existence).

def glob1(dirname, pattern):
    if not dirname:
        dirname = os.curdir
    if isinstance(pattern, _unicode) and not isinstance(dirname, unicode):
        dirname = unicode(dirname, sys.getfilesystemencoding() or
                                   sys.getdefaultencoding())
    try:
        names = os.listdir(dirname)
    except os.error:
        return []
    if pattern[0] != '.':
        names = filter(lambda x: x[0] != '.', names)
    return fnmatch.filter(names, pattern)

def glob0(dirname, basename):
    if basename == '':
        # `os.path.split()` returns an empty basename for paths ending with a
        # directory separator.  'q*x/' should match only directories.
        if os.path.isdir(dirname):
            return [basename]
    else:
        if os.path.lexists(os.path.join(dirname, basename)):
            return [basename]
    return []


magic_check = re.compile('[*?[]')

def has_magic(s):
    return magic_check.search(s) is not None

In [4]:
#Create function to generate array of datetimes at a given interval
def date_range(start_datetime, end_datetime, increment, period):
    result = []
    nxt = start_datetime
    delta = relativedelta(**{period:increment})
    while nxt < end_datetime:
        result.append(nxt)
        nxt += delta
    return result

In [24]:
# Create headernames for files
headernames=["Site","Parameter","Date (LST)","Value","Unit","Instrument",
             "InstrumentNo","QCCode","QC Name","OPCode","OP Name",
             "StationID","Latitude","Longitude","SiteAQS","ParameterAQS",
             "POC","IngestCode","Status","QCIsValid","Qualifiers","Locked"]

# Create array of hourly dates
datearray = pd.DataFrame(np.array(date_range(start_datetime, end_datetime, 1, 'hours')))
datearray.columns = ["Time(GMT-7)"]
# Create copy of date array in same format as BAAQMD dates to use as a date checker later
datecheck = np.array(date_range(start_datetime, end_datetime, 1, 'hours'))
for l in range(len(datecheck)):
    datecheck[l] = datecheck[l].strftime('%Y/%m/%d %H:%M')

# Create list of all files to process
directory='/Users/kaitlynlieschke/Documents/BAAQMD_data/*'
folders=glob(directory)
allfiles=list()
for folder in folders:
    files=glob(os.path.join(folder,"*.csv"))
    allfiles=allfiles+files
    
#Concatenate all files into a dataframe
df=pd.concat((pd.read_csv(file,names=headernames,na_values='-999') for file in allfiles), ignore_index=True)

for k in range(len(baaqmdsites)):
    
    #Create new column for site data in final dataframe populated with NaN
    datearray[baaqmdcodes[k]+"PM(ug/m3)"] = np.nan
    
    #Select data based on site and parameter then reset index
    dfsite=df.loc[(df['Site'] == baaqmdsites[k]) & (df['Parameter'] == "PM25_fem") & (df['QCCode'] == '0')]
    dfsite=dfsite.reset_index()
    
    # Loop through data in datearray to identify dates where measurements occured.
    # Create counter (i) to follow rows in measurement dataframe dfsite
    i = 0
    for j in range(len(datearray)):
        if datecheck[j] == dfsite['Date (LST)'].iloc[i]:
            pmconc = dfsite['Value'].iloc[i]
            datearray.iloc[j, datearray.columns.get_loc(baaqmdcodes[k]+"PM(ug/m3)")] = pmconc
            i =i+1
            if i == len(dfsite):
                break

# #Extract only columns containing date and PM2.5 concentrations and rename columns
# finaldf = pd.concat([dfpm['Date (LST)'],dfpm['Value']], axis=1)
# finaldf.columns = ['Time(GMT-7)', baaqmdcodes[k]+'PM(ug/m3)']

In [27]:
datearray.tail(10)

,Time(GMT-7),BQL_PM(ug/m3),BQS_PM(ug/m3),BQO_PM(ug/m3)
8750,2017-12-31 14:00:00,NaN,NaN,NaN
8751,2017-12-31 15:00:00,NaN,NaN,NaN
8752,2017-12-31 16:00:00,NaN,NaN,NaN
8753,2017-12-31 17:00:00,NaN,NaN,NaN
8754,2017-12-31 18:00:00,NaN,NaN,NaN
8755,2017-12-31 19:00:00,NaN,NaN,NaN
8756,2017-12-31 20:00:00,NaN,NaN,NaN
8757,2017-12-31 21:00:00,NaN,NaN,NaN
8758,2017-12-31 22:00:00,NaN,NaN,NaN
8759,2017-12-31 23:00:00,NaN,NaN,NaN


In [28]:
datearray.to_csv(path_or_buf='baaqmd_coloc2017PM.csv', sep=',', na_rep='NaN', index=False)